In [1]:
fichier = open("C:\\Users\\hp\\Desktop\\flous\\iris.csv")
import csv
lecteur = csv.reader(fichier)
for ligne in lecteur:
    if(lecteur.line_num == 1):
        n = int(ligne[0])
        p = int(ligne[1])
        c = int(ligne[2])
        X = []
        D = []
    else:
        X.append([float(ligne[i]) for i in range(p)])
        D.append(int(ligne[p]))
#print(X[:][0])
import numpy as np
X = np.array(X)

# New Section

In [2]:
import random as rd
import math
class FCM:
    def __init__(self, m, c, X):
        self.m = m
        self.c = c
        self.p = X.shape[1]
        minX = [min(X[:,i]) for i in range(self.p)]
        maxX = [max(X[:,i]) for i in range(self.p)]
        self.V = np.array([[rd.uniform(minX[j], maxX[j]) for j in range(self.p)] for i in range(self.c)])
        self.U = []
        self.Etqt = []
    def calculerU(self, X):
        alpha = -1./(self.m-1.)
        term1 = [ [ pow(sum(np.square(X[k]-self.V[i])), alpha) for k in range(X.shape[0]) ] for i in range(self.c)]
        term2 = [ sum([pow(sum(np.square(X[k]-self.V[i])), alpha) for i in range(self.c)]) for k in range(X.shape[0])]
        self.U = [ [ term1[i][k]/term2[k] for k in range(X.shape[0])] for i in range(self.c)]
    def calculerV(self, X):
        card = [ sum([pow(self.U[i][k], self.m) for k in range(X.shape[0])]) for i in range(self.c)]
        #print(sum(card)) #à cause de la puissance m, la somme est différente de n
        term=[[sum([pow(self.U[i][k],self.m)*X[k][j]for k in range(X.shape[0])])for j in range(self.p)]for i in range(self.c)]
        self.V =[[term[i][j]/card[i] for j in range(self.p)]for i in range(self.c)]
    def PC(self,X):
        res = 0
        for i in range(len(X)):
            for j in range(self.c):
                res += pow(self.U[j][i],2)
            res /= len(X)
        return res  
    def entropie(self,X):
        res = 0
        for i in range(len(X)):
            for j in range(self.c):
                res += self.U[j][i] * math.log2(self.U[j][i])
            res /= len(X)
        return res
    def entrainer(self, X, tmax, eps):
        t = 1
        while(t<tmax):
            self.calculerU(X)
            self.calculerV(X)
            t = t + 1
            self.etiqueterV(X)
           #print(self.tauxErr(X, D)*100./X.shape[0],"%")
    def etiqueterV(self, X):
        card = [0 for i in range(self.c)]
        Vmoy = [ [ 0. for j in range(self.p)] for i in range(self.c)]
        for k in range(X.shape[0]):
            for j in range(self.p):
                Vmoy[D[k]-1][j] = Vmoy[D[k]-1][j] + X[k][j]
        for k in range(X.shape[0]):
            card[D[k]-1] = card[D[k]-1] + 1
        for i in range(self.c):
            for j in range(self.p):
                Vmoy[i][j] = Vmoy[i][j] / card[D[k]-1]
        Vmoy = np.array(Vmoy)
        #print("Vmoy = \n"+str(np.array(Vmoy)))
        dist = [[sum(np.square(self.V[l]-Vmoy[i])) for i in range(self.c)] for l in range(self.c)]
        etqt = [np.argmin(dist[i]) for i in range(self.c)]
        for i in range(self.c):
            self.Etqt.append(etqt[i]+1)
        #print(np.array(self.Etqt))
    def classe(self, vect):
        term1 = [ pow(sum(np.square(vect-self.V[i])), -1./(self.m-1)) for i in range(self.c)]
        term2 = sum([ pow(sum(np.square(vect-self.V[i])), -1./(self.m-1)) for i in range(self.c)])
        u = [term1[i]/term2 for i in range(self.c)]
        return self.Etqt[np.argmax(u)]
    def predict(self,X):
        y_pred = [fcm.classe(x) for x in X]
        return y_pred
    def tauxErr(self, X, D):
        er = 0.
        for k in range(X.shape[0]):
            if(self.classe(X[k]) != D[k]):
                er = er + 1
        return er

# New Section

In [3]:
from sklearn.metrics import confusion_matrix

fcm = FCM(2, c, X)
fcm.calculerU(X)
#for i in range(X.shape[0]):
#print(np.array(fcm.U)[:,i])
fcm.calculerV(X)
print("les prototypes avant l'entrainement :\n",np.array(fcm.V))
print()
fcm.entrainer(X, 1000, 0.001)
print("les prototypes apres l'entrainement :\n",np.array(fcm.V))
# Créer la matrice de confusion en utilisant scikit-learn
confusion = confusion_matrix(D, fcm.predict(X))
print("la matrice de confusion est :\n",confusion)
#print(fcm.tauxErr(X, D)*100./X.shape[0],"%")

les prototypes avant l'entrainement :
 [[5.74469297 3.1712233  3.18644698 0.93766743]
 [6.22746969 2.91519375 4.98225702 1.75031685]
 [5.36557492 3.19053796 2.57924124 0.70146678]]

les prototypes apres l'entrainement :
 [[5.88893236 2.76106936 4.36395164 1.39731504]
 [6.77501122 3.05238227 5.64678178 2.05354666]
 [5.00396596 3.41408886 1.48281553 0.25354632]]
la matrice de confusion est :
 [[50  0  0]
 [ 0 47  3]
 [ 0 13 37]]


In [4]:
import numpy as np
import random as rd
class LVQ:
    def __init__(self, m, c, eta,t_max,X):
        self.m = m
        self.c = c
        self.p = X.shape[1]
        minX = [min(X[:,i]) for i in range(self.p)]
        maxX = [max(X[:,i]) for i in range(self.p)]
        self.V = np.array([ [ rd.uniform(minX[j], maxX[j]) for j in range(self.p)] for i in range(self.c)])
        self.eta=eta
        self.t_max=t_max
    def gagnant(self,x):
        g=np.argmin([sum(np.square(x-self.V[i]))for i in range(self.c)])
        return g
    def ajuster(self,x,g):
        self.V[g]+=self.eta*(x-self.V[g])
    def etiqueterV(self, X):
        card = [0 for i in range(self.c)]
        Vmoy = [ [ 0. for j in range(self.p)] for i in range(self.c)]
        for k in range(X.shape[0]):
            for j in range(self.p):
                Vmoy[D[k]-1][j] = Vmoy[D[k]-1][j] + X[k][j]
        for k in range(X.shape[0]):
            card[D[k]-1] = card[D[k]-1] + 1
        for i in range(self.c):
            for j in range(self.p):
                Vmoy[i][j] = Vmoy[i][j] / card[D[k]-1]
        Vmoy = np.array(Vmoy)
        #print("Vmoy = \n"+str(np.array(Vmoy)))
        dist = [[sum(np.square(self.V[l]-Vmoy[i])) for i in range(self.c)] for l in range(self.c)]
        etqt = [np.argmin(dist[i]) for i in range(self.c)]
        for i in range(self.c):
            self.Etqt.append(etqt[i]+1)
        #print(np.array(self.Etqt))
    def classe(self, vect):
        term1 = [ pow(sum(np.square(vect-self.V[i])), -1./(self.m-1)) for i in range(self.c)]
        term2 = sum([ pow(sum(np.square(vect-self.V[i])), -1./(self.m-1)) for i in range(self.c)])
        u = [term1[i]/term2 for i in range(self.c)]
        return self.Etqt[np.argmax(u)]
    def entrainer(self,X):
        for t in range(self.t_max):
            for k in range(X.shape[0]):
                self.ajuster(X[k],self.gagnant(X[k]))
    def predict(self,X):
        y_pred = [fcm.classe(x) for x in X]
        return y_pred
    def afficher(self):
        print(self.V)
    '''
    def desision(self,X):
        decision=[]
        for i in range(X.shape[0]):
            decision.append(np.argmin([sum(np.square(X[i]-self.V[j])) for j in range(self.c)])+1)
        return decision
    '''

In [5]:
from sklearn.metrics import confusion_matrix
lvq=LVQ(2,3,0.2,1000,X)
print('les prototypes avant l entrainement :')
lvq.afficher()
lvq.entrainer(X)
print('les prototypes apres l entrainement :')
lvq.afficher()
# Créer la matrice de confusion en utilisant scikit-learn
confusion = confusion_matrix(D,lvq.predict(X))
print("la matrice de confusion est :\n",confusion)


les prototypes avant l entrainement :
[[5.34913176 3.77722477 5.53062885 1.04117396]
 [5.81380954 2.71138143 5.60169504 1.69605946]
 [7.51382915 3.1722541  1.71207155 1.47097415]]
les prototypes apres l entrainement :
[[6.62946492 3.16403886 5.47517107 2.23081586]
 [6.00351005 2.76524099 4.98439872 1.74671368]
 [4.9552498  3.39914833 1.47075856 0.2477971 ]]
la matrice de confusion est :
 [[50  0  0]
 [ 0 47  3]
 [ 0 13 37]]


In [6]:

import numpy as np
class FLVQ:
    def __init__(self, c, eps, tmax, X, eta):
        self.c = c
        self.eta = eta
        self.eps = eps
        self.tmax = tmax
        self.min = [min(X[:,j]) for j in range(X.shape[1])]
        self.max = [max(X[:,j]) for j in range(X.shape[1])]
        self.V = [[np.random.uniform(self.min[j], self.max[j]) for j in range(X.shape[1])] for i in range(c)]
        self.W = np.random.rand(X.shape[0], c)
    def gagnant(self,x):
        g=np.argmin([sum(np.square(x-self.V[i]))for i in range(self.c)])
        return g  
    def Ajuster(self, x , g, t):
        d = np.array([np.linalg.norm(x - self.V[i]) for i in range(self.c)])
        s = np.sum(d / d[g])
        self.W[t, :] = np.power((d / d[g]), 2) / s
        for i in range(self.c):
            self.V[i] += self.eta * self.W[t,i] * (x - self.V[i])
    def etiqueterV(self, X):
        card = [0 for i in range(self.c)]
        Vmoy = [ [ 0. for j in range(self.p)] for i in range(self.c)]
        for k in range(X.shape[0]):
            for j in range(self.p):
                Vmoy[D[k]-1][j] = Vmoy[D[k]-1][j] + X[k][j]
        for k in range(X.shape[0]):
            card[D[k]-1] = card[D[k]-1] + 1
        for i in range(self.c):
            for j in range(self.p):
                Vmoy[i][j] = Vmoy[i][j] / card[D[k]-1]
        Vmoy = np.array(Vmoy)
        #print("Vmoy = \n"+str(np.array(Vmoy)))
        dist = [[sum(np.square(self.V[l]-Vmoy[i])) for i in range(self.c)] for l in range(self.c)]
        etqt = [np.argmin(dist[i]) for i in range(self.c)]
        for i in range(self.c):
            self.Etqt.append(etqt[i]+1)
        #print(np.array(self.Etqt))
    def classe(self, vect):
        term1 = [ pow(sum(np.square(vect-self.V[i])), -1./(self.m-1)) for i in range(self.c)]
        term2 = sum([ pow(sum(np.square(vect-self.V[i])), -1./(self.m-1)) for i in range(self.c)])
        u = [term1[i]/term2 for i in range(self.c)]
        return self.Etqt[np.argmax(u)]
    """
    def entrainer(self,X):
        for t in range(self.t_max):
            for k in range(X.shape[0]):
                self.ajuster(X[k],self.gagnant(X[k]))
    """
    def predict(self,X):
        y_pred = [fcm.classe(x) for x in X]
        return y_pred
    def entrainer(self, X):
        n = X.shape[0]
        for t in range(self.tmax):
            for k in range(n):
                x = X[k]
                g = self.gagnant(x)
                self.Ajuster(x, g, k)
    def afficher(self):
        print(self.V)
    def desision(self,X):
        decision=[]
        for i in range(X.shape[0]):
            decision.append(np.argmin([sum(np.square(X[i]-self.V[j])) for j in range(self.c)])+1)
        return decision

In [7]:
from sklearn.metrics import confusion_matrix
flvq=FLVQ(3,0.01,1000,X,0.2)
print('les prototypes avant l entrainement :')
flvq.afficher()
flvq.entrainer(X)
print('les prototypes apres l entrainement :')
flvq.afficher()
# Créer la matrice de confusion en utilisant scikit-learn
confusion = confusion_matrix(D,flvq.desision(X))
print("la matrice de confusion est :\n",confusion)


les prototypes avant l entrainement :
[[4.679391650267964, 4.277000171330027, 1.8648711205945818, 1.340955423850949], [7.195797227161693, 3.5718766308283834, 5.6139138344968655, 1.3420411585849943], [4.474531273986358, 2.0448341899064695, 3.2238391976209035, 0.43789923928043584]]
les prototypes apres l entrainement :
[array([6.50704615, 3.00525013, 5.39164141, 2.02895129]), array([6.50704615, 3.00525013, 5.39164141, 2.02895129]), array([6.50704615, 3.00525013, 5.39164141, 2.02895129])]
la matrice de confusion est :
 [[50  0  0]
 [50  0  0]
 [50  0  0]]


In [8]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

def entropie(labels):
    _, counts = np.unique(labels, return_counts=True)
    probs = counts / len(labels)
    return -np.sum(probs * np.log2(probs))

def evaluate_algorithm(algo, X, y):
    algo_name = algo.__class__.__name__
    if algo==fcm:
        pc=fcm.PC(X)
        ent=fcm.entropie(X)
        print(f"{algo_name} : PC = {pc:.3f}, Entropie = {ent:.3f}")
    else:
        predictions = [algo.gagnant(x) for x in X]
        pc = accuracy_score(y, predictions)
        ent = entropie(predictions)
        print(f"{algo_name} : PC = {pc:.3f}, Entropie = {ent:.3f}")
    

In [9]:
evaluate_algorithm(fcm, X, D)
evaluate_algorithm(lvq, X, D)
evaluate_algorithm(flvq, X, D)

FCM : PC = 0.003, Entropie = -0.008
LVQ : PC = 0.007, Entropie = 1.549
FLVQ : PC = 0.000, Entropie = -0.000


# New Section